In [2]:
import pandas as pd
from core.enums import Direction, SurrogateMethod, GeneralCriterion, FlowType
from core.constraint import Constraint, Relation
from modular_parts.weights import surrogate_weights
from modular_parts.preference import compute_reinforced_preference,    \
    compute_veto, compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows, \
    calculate_net_outranking_flows
from modular_parts.ranking import calculate_promethee_ii_ranking
from modular_parts.choice import compute_decision

criteria = ['g1', 'g2', 'g3']
ranks = [2, 3, 1]
criteria_ranks = pd.Series(ranks, criteria)
weights = surrogate_weights(criteria_ranks, SurrogateMethod.ROC)
weights

g1    0.278
g2    0.111
g3    0.611
Name: weights, dtype: float64

In [3]:
directions = [Direction.MAX, Direction.MAX, Direction.MIN]
q = [0, 0, 100]
p = [0, 1, 300]
rp = [10, 5, 500]
omega = [1.3, 1.2, 1.5]
gc = [GeneralCriterion.V_SHAPE_INDIFFERENCE,
      GeneralCriterion.V_SHAPE_INDIFFERENCE,
      GeneralCriterion.V_SHAPE_INDIFFERENCE]

criteria_directions = pd.Series(directions, criteria)
indifference_thresholds = pd.Series(q, criteria)
preference_thresholds = pd.Series(p, criteria)
reinforced_preference_thresholds = pd.Series(rp, criteria)
reinforcement_factors = pd.Series(omega, criteria)
generalised_criteria = pd.Series(gc, criteria)

s_parameters = pd.Series([None, None, None], criteria)

alternatives = ['ITA', 'BEL', 'GER', 'SWE', 'AUT', 'FRA']
performances = [[98, 8, 400],
                [58, 0, 800],
                [66, 5, 1000],
                [74, 3, 600],
                [80, 7, 200],
                [82, 10, 600]]
protifles = [[80, 7, 200],
             [82, 10, 600]]

alternatives_performances = pd.DataFrame(data=performances, columns=criteria,
                                         index=alternatives)
protifles_perfomance = pd.DataFrame(data=protifles, columns=criteria, index = ['prof1','prof2'])
pref, part_pref = compute_reinforced_preference(alternatives_performances,
                                                preference_thresholds,
                                                indifference_thresholds,
                                                s_parameters,
                                                generalised_criteria,
                                                criteria_directions,
                                                reinforced_preference_thresholds,
                                                reinforcement_factors,
                                                weights)
part_pref

ITA  BEL  GER  SWE  AUT  FRA
criteria                                  
g1       ITA  0.0  1.3  1.3  1.3  1.3  1.3
         BEL  0.0  0.0  0.0  0.0  0.0  0.0
         GER  0.0  1.0  0.0  0.0  0.0  0.0
         SWE  0.0  1.3  1.0  0.0  0.0  0.0
         AUT  0.0  1.3  1.3  1.0  0.0  0.0
         FRA  0.0  1.3  1.3  1.0  1.0  0.0
g2       ITA  0.0  1.2  1.0  1.0  1.0  0.0
         BEL  0.0  0.0  0.0  0.0  0.0  0.0
         GER  0.0  1.0  0.0  1.0  0.0  0.0
         SWE  0.0  1.0  0.0  0.0  0.0  0.0
         AUT  0.0  1.2  1.0  1.0  0.0  0.0
         FRA  1.0  1.2  1.0  1.2  1.0  0.0
g3       ITA  0.0  1.0  1.5  0.5  0.0  0.5
         BEL  0.0  0.0  0.5  0.0  0.0  0.0
         GER  0.0  0.0  0.0  0.0  0.0  0.0
         SWE  0.0  0.5  1.0  0.0  0.0  0.0
         AUT  0.5  1.5  1.5  1.0  0.0  1.0
         FRA  0.0  0.5  1.0  0.0  0.0  0.0

In [4]:
part_pref.loc["g3"].T

,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.0,0.0,0.0,0.0,0.5,0.0
BEL,1.0,0.0,0.0,0.5,1.5,0.5
GER,1.5,0.5,0.0,1.0,1.5,1.0
SWE,0.5,0.0,0.0,0.0,1.0,0.0
AUT,0.0,0.0,0.0,0.0,0.0,0.0
FRA,0.5,0.0,0.0,0.0,1.0,0.0


In [5]:
pref

,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.000,1.000,1.000,0.718,0.436,0.616
BEL,0.000,0.000,0.306,0.000,0.000,0.000
GER,0.000,0.389,0.000,0.111,0.000,0.000
SWE,0.000,0.718,0.889,0.000,0.000,0.000
AUT,0.306,1.000,1.000,1.000,0.000,0.611
FRA,0.111,0.724,1.000,0.402,0.389,0.000


In [8]:
veto = [None, 5, 400]
veto_thresholds = pd.Series(veto, criteria)

veto, partial_veto = compute_veto(alternatives_performances, weights,
                                  veto_thresholds, criteria_directions, profiles_performance=protifles_perfomance)
partial_veto

(              prof1  prof2
 criteria                  
 g1       ITA      0      0
          BEL      0      0
          GER      0      0
          SWE      0      0
          AUT      0      0
          FRA      0      0
 g2       ITA      0      0
          BEL      1      1
          GER      0      1
          SWE      0      1
          AUT      0      0
          FRA      0      0
 g3       ITA      0      0
          BEL      1      0
          GER      1      1
          SWE      1      0
          AUT      0      0
          FRA      1      0,
                 ITA  BEL  GER  SWE  AUT  FRA
 criteria                                    
 g1       prof1    0    0    0    0    0    0
          prof2    0    0    0    0    0    0
 g2       prof1    0    0    0    0    0    0
          prof2    0    0    0    0    0    0
 g3       prof1    0    0    0    0    0    0
          prof2    0    0    0    0    1    0)

In [9]:
veto

(     prof1  prof2
 ITA      0      0
 BEL      1      1
 GER      1      1
 SWE      1      1
 AUT      0      0
 FRA      1      0,
        ITA  BEL  GER  SWE  AUT  FRA
 prof1    0    0    0    0    0    0
 prof2    0    0    0    0    1    0)

In [7]:
_, _, overall_pref = compute_veto(alternatives_performances, weights,
                                  veto_thresholds, criteria_directions,
                                  preferences=pref)
overall_pref

,ITA,BEL,GER,SWE,AUT,FRA
ITA,0.000,1.000,1.000,0.718,0.436,0.616
BEL,0.000,0.000,0.000,0.000,0.000,0.000
GER,0.000,0.389,0.000,0.000,0.000,0.000
SWE,0.000,0.718,0.889,0.000,0.000,0.000
AUT,0.306,1.000,1.000,1.000,0.000,0.611
FRA,0.111,0.724,1.000,0.402,0.000,0.000


In [8]:
outranking_flows = calculate_promethee_outranking_flows(overall_pref,
                                                        FlowType.BASIC)
outranking_flows

,positive,negative
ITA,0.7540,0.0834
BEL,0.0000,0.7662
GER,0.0778,0.7778
SWE,0.3214,0.4240
AUT,0.7834,0.0872
FRA,0.4474,0.2454


In [9]:
pIIflows = calculate_net_outranking_flows(outranking_flows, True)
pIIflows

,positive,negative,net
ITA,0.7540,0.0834,0.6706
BEL,0.0000,0.7662,-0.7662
GER,0.0778,0.7778,-0.7000
SWE,0.3214,0.4240,-0.1026
AUT,0.7834,0.0872,0.6962
FRA,0.4474,0.2454,0.2020


In [10]:
calculate_promethee_ii_ranking(pIIflows)

1    AUT
2    ITA
3    FRA
4    SWE
5    GER
6    BEL
Name: ranking, dtype: object

In [11]:
cost = [22, 17, 25, 28, 20, 18]
constraint1 = Constraint(cost, Relation.LEQ, 40)
x = [1, 1, 1, 1, 1, 1]
constraint2 = Constraint(x, Relation.LEQ, 2)
compute_decision(pIIflows['net'], [constraint1, constraint2])

0    AUT
1    FRA
Name: chosen alternatives, dtype: object

In [12]:
from modular_parts.preference import compute_discordance

p, x = compute_discordance(criteria, part_pref, 2)
x

ITA  BEL  GER  SWE  AUT  FRA
g1 ITA  0.0  0.0  0.0  0.0  0.0  0.0
   BEL  1.3  0.0  1.0  1.3  1.3  1.3
   GER  1.3  0.0  0.0  1.0  1.3  1.3
   SWE  1.3  0.0  0.0  0.0  1.0  1.0
   AUT  1.3  0.0  0.0  0.0  0.0  1.0
   FRA  1.3  0.0  0.0  0.0  0.0  0.0
g2 ITA  0.0  0.0  0.0  0.0  0.0  1.0
   BEL  1.2  0.0  1.0  1.0  1.2  1.2
   GER  1.0  0.0  0.0  0.0  1.0  1.0
   SWE  1.0  0.0  1.0  0.0  1.0  1.2
   AUT  1.0  0.0  0.0  0.0  0.0  1.0
   FRA  0.0  0.0  0.0  0.0  0.0  0.0
g3 ITA  0.0  0.0  0.0  0.0  0.5  0.0
   BEL  1.0  0.0  0.0  0.5  1.5  0.5
   GER  1.5  0.5  0.0  1.0  1.5  1.0
   SWE  0.5  0.0  0.0  0.0  1.0  0.0
   AUT  0.0  0.0  0.0  0.0  0.0  0.0
   FRA  0.5  0.0  0.0  0.0  1.0  0.0

In [13]:
x = pd.DataFrame(index=["a", "b", "c"], columns=["a", "b"], data=[[0, 1],
                                                                  [0, 1],
                                                                  [0, 1]])
x.T

,a,b,c
a,0,0,0
b,1,1,1


In [14]:
x

,a,b
a,0,1
b,0,1
c,0,1
